# RetinaFace 
https://github.com/biubug6/Pytorch_Retinaface <br>
https://github.com/bubbliiiing/retinaface-tf2

## 下載 wider face dataset

In [ ]:
!pip install -q gdown

In [ ]:
!gdown https://drive.google.com/uc?id=11UGV3nbVv1x9IC--_tK3Uxf7hA6rlbsS -O data/widerface.zip

In [ ]:
!cd data; unzip widerface.zip
!rm data/widerface.zip

## 導入套件

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nets.retinaface import Retinaface
from nets.retinanet_training import Generator
from nets.retinanet_training import conf_loss, box_smooth_l1, ldm_smooth_l1
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from utils.utils import Anchors, BBoxUtility
from utils.config import cfg_mnet

In [2]:
training_dataset_path = './data/widerface/train/label.txt'
cfg = cfg_mnet

# 根據 cfg 獲得 anchors 與 bbox_util 工具箱
anchors = Anchors(cfg).get_anchors()
bbox_util = BBoxUtility(anchors)

In [3]:
# 訓練參數設置
checkpoint = ModelCheckpoint('model_data/best_model.h5',
                             monitor='loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=2, verbose=1)
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=6, verbose=1)

In [4]:
batch_size = 32
learning_rate = 1e-4
epochs = 100

In [5]:
# 根據 cfg 創建 Retinaface network
model = Retinaface(cfg)

In [6]:
model.compile(loss={
                    'bbox_reg'  : box_smooth_l1(),
                    'cls'       : conf_loss(),
                    'landm_reg' : ldm_smooth_l1()},
              optimizer=keras.optimizers.Adam(lr=learning_rate)
        )

In [7]:
# 獲得 data 產生器
gen = Generator(training_dataset_path, cfg['image_size'], batch_size, bbox_util)

FileNotFoundError: [Errno 2] No such file or directory: './data/widerface/train/label.txt'

In [ ]:
model.fit(gen.generate(False), 
          steps_per_epoch=gen.get_len()//batch_size,
          verbose=1,
          epochs=epochs,
          callbacks=[checkpoint, reduce_lr, early_stopping])

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 159 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 159 all-reduces with algorithm = nccl, num_packs = 1
25/25 [==============================] - 564s 23s/step - cls_loss: 5.7941 - loss: 28.8047 - landm_reg_loss: 16.3538 - bbox_reg_loss: 6.6568 - lr: 0.0100
Epoch 2/100
25/25 [==============================] - 570s 23s/step - cls_loss: 3.1816 - loss: 23.3520 - landm_reg_loss: 14.7720 - bbox_reg_loss: 5.3984 - lr: 0.0100
Epoch 3/100
25/25 [==============================] - 566s 23s/step - cls_loss: 3.0944 - loss: 22.1591 - landm_reg_loss: 14.0454 - bbox_reg_loss: 5.0193 - lr: 0.0100
Epoch 4/100
25/25 [==============================] - 567s 23s/step - cls_loss: 3.0542 - loss: 20.8801 - landm_reg_loss: 13.1047 - bbox_reg_loss: 4.7212 - lr: 0.0100
Epoch 5/100
25/25 [==============================] - 567s 23s/step - cls_loss: 2.9875 - loss: 19.8955 - landm_reg_loss: 12.3633 - bbox_reg_loss: 4.5447 - lr: 0.0100
E

In [8]:
# 將最好的 model 權重讀進去
model = Retinaface(cfg)
model.load_weights('logs/best_model.h5')

In [20]:
!pip install -q onnx onnxmltools

Note: you may need to restart the kernel to use updated packages.


In [8]:
import onnx
import onnxmltools
from tensorflow.keras.models import load_model

In [9]:
onnx_path = 'retinaface-320.onnx'
onnx_model = onnxmltools.convert_keras(model)

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 440 -> 129
The maximum opset needed by this model is only 11.


### 將 Dynamic 的部分改為 固定 batch size 為 1

In [10]:
onnx_model.graph.input[0].type.tensor_type.shape.dim

[dim_param: "N"
, dim_value: 320
, dim_value: 320
, dim_value: 3
]

In [11]:
onnx_model.graph.input[0].type.tensor_type.shape.dim[0].dim_value = 1

In [12]:
onnx_model.graph.output[0].type.tensor_type.shape.dim

[dim_param: "N"
, dim_param: "M1"
, dim_value: 4
]

In [13]:
onnx_model.graph.output[1].type.tensor_type.shape.dim

[dim_param: "N"
, dim_param: "M2"
, dim_value: 2
]

In [14]:
for output in onnx_model.graph.output:
    output.type.tensor_type.shape.dim[0].dim_value = 1
    output.type.tensor_type.shape.dim[1].dim_value = anchors.shape[0]

In [15]:
onnx.checker.check_model(onnx_model)
onnx.save(onnx_model, onnx_path)